In [1]:
from quantfreedom.enums import *
from quantfreedom.helper_funcs import dl_ex_candles
from quantfreedom.simulate import run_df_backtest, or_backtest
from quantfreedom.strategies.ex_strat_004 import RSIRisingFalling
import numpy as np

np.set_printoptions(formatter={"float_kind": "{:0.2f}".format})

%load_ext autoreload
%autoreload 2

In [2]:
candles = dl_ex_candles(
    exchange="mufex",
    symbol="BTCUSDT",
    timeframe="1h",
    candles_to_dl=3000,
)

timestamps = candles[:, CandleBodyType.Timestamp]
datetimes = timestamps.astype("datetime64[ms]")
open = candles[:, CandleBodyType.Open]
high = candles[:, CandleBodyType.High]
low = candles[:, CandleBodyType.Low]
close = candles[:, CandleBodyType.Close]
volume = candles[:, CandleBodyType.Volume]

In [3]:
short_strat = RSIRisingFalling(
    long_short="short",
    rsi_length=np.array([14]),
    rsi_is_above=np.arange(50, 76, 5),
)
short_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
short_strat.plot_signals(candles=candles)

In [4]:
long_strat = RSIRisingFalling(
    long_short="long",
    rsi_length=np.array([14]),
    rsi_is_below=np.arange(35, 61, 5),
)
long_strat.set_entries_exits_array(
    candles=candles,
    ind_set_index=2,
)
long_strat.plot_signals(candles=candles)

In [5]:
backtest_settings = BacktestSettings()

exchange_settings = ExchangeSettings(
    asset_tick_step=3,
    leverage_mode=1,
    leverage_tick_step=2,
    limit_fee_pct=0.0003,
    market_fee_pct=0.0006,
    max_asset_size=100.0,
    max_leverage=150.0,
    min_asset_size=0.001,
    min_leverage=1.0,
    mmr_pct=0.004,
    position_mode=3,
    price_tick_step=1,
)

dos_arrays = DynamicOrderSettingsArrays(
    max_equity_risk_pct=np.array([12]),
    max_trades=np.array([5]),
    risk_account_pct_size=np.array([3]),
    risk_reward=np.array([2, 5]),
    sl_based_on_add_pct=np.array([0.1, 0.25, 0.5]),
    sl_based_on_lookback=np.array([20, 50]),
    sl_bcb_type=np.array([CandleBodyType.Low]),
    sl_to_be_cb_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct=np.array([0]),
    trail_sl_bcb_type=np.array([CandleBodyType.Low]),
    trail_sl_by_pct=np.array([0.5, 1.0]),
    trail_sl_when_pct=np.array([1, 2]),
)

static_os = StaticOrderSettings(
    increase_position_type=IncreasePositionType.RiskPctAccountEntrySize,
    leverage_strategy_type=LeverageStrategyType.Dynamic,
    pg_min_max_sl_bcb="min",
    sl_strategy_type=StopLossStrategyType.SLBasedOnCandleBody,
    sl_to_be_bool=False,
    starting_bar=50,
    starting_equity=1000.0,
    static_leverage=None,
    tp_fee_type="limit",
    tp_strategy_type=TakeProfitStrategyType.RiskReward,
    trail_sl_bool=False,
    z_or_e_type=None,
)

In [6]:
backtest_results = run_df_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    logger_bool=False,
)

Starting the backtest now ... and also here are some stats for your backtest.

Total indicator settings to test: 6
Total order settings to test: 48
Total combinations of settings to test: 288
Total candles: 3,000
Total candles to test: 864,000


In [7]:
backtest_results.sort_values(by=["qf_score"], ascending=False).head(20)

,ind_set_idx,dos_index,total_trades,wins,losses,gains_pct,win_rate,qf_score,fees_paid,total_pnl,ending_eq
209,4,17,36.0,17,19,375.681,47.222,0.891,555.355,3756.811,4756.811
210,4,18,36.0,17,19,375.681,47.222,0.891,555.355,3756.811,4756.811
211,4,19,36.0,17,19,375.681,47.222,0.891,555.355,3756.811,4756.811
208,4,16,36.0,17,19,375.681,47.222,0.891,555.355,3756.811,4756.811
180,3,36,16.0,7,9,689.232,43.750,0.889,425.142,6892.323,7892.323
183,3,39,16.0,7,9,689.232,43.750,0.889,425.142,6892.323,7892.323
182,3,38,16.0,7,9,689.232,43.750,0.889,425.142,6892.323,7892.323
181,3,37,16.0,7,9,689.232,43.750,0.889,425.142,6892.323,7892.323
287,5,47,9.0,4,5,247.405,44.444,0.879,60.787,2474.048,3474.048
284,5,44,9.0,4,5,247.405,44.444,0.879,60.787,2474.048,3474.048


In [8]:
order_records_df = or_backtest(
    backtest_settings=backtest_settings,
    candles=candles,
    dos_arrays=dos_arrays,
    exchange_settings=exchange_settings,
    static_os=static_os,
    strategy=long_strat,
    dos_index=17,
    ind_set_index=4,
    plot_results=True,
    logger_bool=True,
)

DynamicOrderSettings(
    max_equity_risk_pct = 0.12,
    max_trades = 5,
    risk_account_pct_size = 0.03,
    risk_reward = 2.0,
    sl_based_on_add_pct = 0.005,
    sl_based_on_lookback = 20,
    sl_bcb_type = 3,
    sl_to_be_cb_type = 6,
    sl_to_be_when_pct = 0.0,
    trail_sl_bcb_type = 3,
    trail_sl_by_pct = 0.005,
    trail_sl_when_pct = 0.02,
)
IndicatorSettingsArrays(
    rsi_is_above = nan,
    rsi_is_below = 55.0,
    rsi_length = 14,
)


In [9]:
order_records_df

,ind_set_idx,or_set_idx,bar_idx,timestamp,datetime,order_status,equity,available_balance,cash_borrowed,cash_used,...,entry_size_usd,entry_price,exit_price,position_size_asset,position_size_usd,realized_pnl,sl_pct,sl_price,tp_pct,tp_price
0,4,17,52,1692471600000,2023-08-19 19:00:00,EntryFilled,1000.000,962.121,1590.720,37.879,...,1628.599,26109.8,NaN,0.062,1628.599,NaN,1.7,25659.9,3.8,27095.5
1,4,17,55,1692482400000,2023-08-19 22:00:00,EntryFilled,1000.000,884.423,4872.319,115.577,...,1768.577,26071.1,NaN,0.130,3397.176,NaN,1.7,25659.9,3.6,27042.1
2,4,17,57,1692489600000,2023-08-20 00:00:00,EntryFilled,1000.000,769.459,9692.552,230.541,...,1653.598,26088.3,NaN,0.193,5050.774,NaN,1.7,25659.9,3.7,27048.2
3,4,17,60,1692500400000,2023-08-20 03:00:00,EntryFilled,1000.000,615.857,16079.159,384.143,...,1719.976,26081.8,NaN,0.259,6770.750,NaN,1.7,25659.9,3.6,27040.5
4,4,17,126,1692738000000,2023-08-22 21:00:00,StopLossFilled,880.069,880.069,NaN,NaN,...,NaN,NaN,25659.9,NaN,NaN,-119.931,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,4,17,2771,1702260000000,2023-12-11 02:00:00,StopLossFilled,4756.811,4756.811,NaN,NaN,...,NaN,NaN,42612.1,NaN,NaN,-642.987,NaN,NaN,NaN,NaN
159,4,17,2773,1702267200000,2023-12-11 04:00:00,EntryFilled,4756.811,4602.549,2481.781,154.262,...,2636.043,42312.9,NaN,0.062,2636.043,NaN,5.3,40083.0,10.9,46911.0
160,4,17,2776,1702278000000,2023-12-11 07:00:00,EntryFilled,4756.811,4287.040,7495.120,469.771,...,2847.067,42135.2,NaN,0.130,5483.110,NaN,5.1,40083.0,10.5,46668.3
161,4,17,2778,1702285200000,2023-12-11 09:00:00,EntryFilled,4756.811,3821.330,14679.724,935.481,...,2636.975,42246.3,NaN,0.192,8120.085,NaN,5.1,40083.0,10.6,46723.1
